## Obtencion de diccionarios
Generar un diccionario booleano por cada archivo txt en formato json para evitar mantener todo en memoria

In [3]:
import os
import re
import json
import pandas as pd

In [10]:


# Función para procesar un archivo de texto y generar el mapeo de palabras
def generar_mapeo(archivo):
    mapeo = {}
    with open(archivo, 'r', encoding='utf-8') as file:
        texto = file.read()
        # Utilizar expresiones regulares para obtener palabras
        palabras = re.findall(r'\b\w+\b', texto.lower())
        # Actualizar el mapeo con las palabras del archivo
        for palabra in palabras:
            mapeo[palabra] = True
    return mapeo

In [11]:

# Obtener la lista de archivos de texto en un directorio
directorio_entrada = r'C:\Users\erick\Escritorio\RI\24a\data'
directorio_salida = r'C:\Users\erick\Escritorio\RI\24a\json'


In [12]:

# Crear el directorio de salida si no existe
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Procesar cada archivo de texto y generar el archivo JSON correspondiente
archivos_txt = [f for f in os.listdir(directorio_entrada) if f.endswith('.txt')]
for archivo_txt in archivos_txt:
    mapeo_palabras = generar_mapeo(os.path.join(directorio_entrada, archivo_txt))
    nombre_json = os.path.splitext(archivo_txt)[0] + '_mapeo.json'
    ruta_json = os.path.join(directorio_salida, nombre_json)
    with open(ruta_json, 'w', encoding='utf-8') as json_file:
        json.dump(mapeo_palabras, json_file, ensure_ascii=False, indent=4)



In [15]:
# Directorio donde se encuentran los archivos JSON generados
directorio_json = r'C:\Users\erick\Escritorio\RI\24a\json'

# Obtener la lista de archivos JSON
archivos_json = [f for f in os.listdir(directorio_json) if f.endswith('_mapeo.json')]

# Diccionario para almacenar las palabras y su información
diccionario_unificado = {}

# Leer cada archivo JSON y agregar sus datos al diccionario unificado
for archivo_json in archivos_json:
    ruta_json = os.path.join(directorio_json, archivo_json)
    with open(ruta_json, 'r', encoding='utf-8') as json_file:
        datos_json = json.load(json_file)
        # Agregar las palabras y su información al diccionario unificado
        for palabra, valor in datos_json.items():
            if palabra not in diccionario_unificado:
                diccionario_unificado[palabra] = {
                    'en_archivos': [archivo_json],  # Lista de archivos donde se encuentra la palabra
                    'aparece': valor  # Valor booleano de aparición
                }
            else:
                diccionario_unificado[palabra]['en_archivos'].append(archivo_json)

# Guardar el diccionario unificado en un archivo JSON
with open('diccionario_unificado.json', 'w', encoding='utf-8') as json_file:
    json.dump(diccionario_unificado, json_file, ensure_ascii=False, indent=4)

In [16]:
# Leer el archivo JSON con la información de palabras y archivos
with open('diccionario_unificado.json', 'r', encoding='utf-8') as json_file:
    diccionario_unificado = json.load(json_file)

# Obtener la clave que representa los datos de palabras y archivos
datos_clave = next(iter(diccionario_unificado.keys()))

# Obtener los nombres de los archivos en orden
nombres_archivos = diccionario_unificado[datos_clave]["en_archivos"]

# Crear el DataFrame con la primera fila de nombres de archivos
df = pd.DataFrame(columns=["nombre de archivo"] + nombres_archivos)

# Iterar sobre las palabras y agregar filas al DataFrame
for palabra, info in diccionario_unificado.items():
    if palabra != datos_clave:  # Ignorar la información inicial
        fila = [palabra] + [archivo in info["en_archivos"] for archivo in nombres_archivos]
        df.loc[len(df)] = fila

# Imprimir el DataFrame resultante
print(df)


       nombre de archivo  pg100_mapeo.json  pg10676_mapeo.json  \
0                project              True                True   
1              gutenberg              True                True   
2                  ebook              True                True   
3                     of              True                True   
4               complete              True                True   
...                  ...               ...                 ...   
199515            debits             False               False   
199516           quixana             False               False   
199517              p74b             False               False   
199518    tordesillesque             False               False   
199519              p74e             False               False   

        pg10681_mapeo.json  pg1080_mapeo.json  pg1184_mapeo.json  \
0                     True               True               True   
1                     True               True               True   
2  

In [17]:
print(df.head(10))

  nombre de archivo  pg100_mapeo.json  pg10676_mapeo.json  pg10681_mapeo.json  \
0           project              True                True                True   
1         gutenberg              True                True                True   
2             ebook              True                True                True   
3                of              True                True                True   
4          complete              True                True                True   
5             works              True                True                True   
6           william              True               False               False   
7       shakespeare              True               False                True   
8              this              True                True                True   
9                is              True                True                True   

   pg1080_mapeo.json  pg1184_mapeo.json  pg11_mapeo.json  pg120_mapeo.json  \
0               True          

In [4]:
df.to_csv('matriz.csv', index=False)  # index=False evita guardar el índice del DataFrame como una columna adicional en el archivo CSV

NameError: name 'df' is not defined

In [6]:
# Cargar el DataFrame desde un archivo CSV
df = pd.read_csv('matriz.csv')

In [7]:
def buscar_palabra(palabra, dataframe):
    # Obtener las filas que contienen la palabra buscada
    filas_con_palabra = dataframe[dataframe.iloc[:, 0] == palabra]
    # Filtrar las columnas que tienen valores verdaderos en al menos una fila
    columnas_con_verdaderos = filas_con_palabra.loc[:, filas_con_palabra.any()].columns.tolist()
    return columnas_con_verdaderos


In [8]:
palabra_buscada = input("Ingrese la palabra a buscar: ")
resultados_busqueda = buscar_palabra(palabra_buscada, df)
print(resultados_busqueda)

['nombre de archivo', 'pg100_mapeo.json', 'pg10676_mapeo.json', 'pg10681_mapeo.json', 'pg1184_mapeo.json', 'pg11_mapeo.json', 'pg120_mapeo.json', 'pg1259_mapeo.json', 'pg1260_mapeo.json', 'pg1342_mapeo.json', 'pg1400_mapeo.json', 'pg145_mapeo.json', 'pg1513_mapeo.json', 'pg16389_mapeo.json', 'pg16594_mapeo.json', 'pg1661_mapeo.json', 'pg16_mapeo.json', 'pg1727_mapeo.json', 'pg174_mapeo.json', 'pg1952_mapeo.json', 'pg19694_mapeo.json', 'pg19926_mapeo.json', 'pg1998_mapeo.json', 'pg205_mapeo.json', 'pg2160_mapeo.json', 'pg21700_mapeo.json', 'pg21765_mapeo.json', 'pg219_mapeo.json', 'pg23_mapeo.json', 'pg24238_mapeo.json', 'pg244_mapeo.json', 'pg25344_mapeo.json', 'pg2542_mapeo.json', 'pg2554_mapeo.json', 'pg2581_mapeo.json', 'pg2591_mapeo.json', 'pg2600_mapeo.json', 'pg2641_mapeo.json', 'pg2701_mapeo.json', 'pg27827_mapeo.json', 'pg28054_mapeo.json', 'pg2814_mapeo.json', 'pg2852_mapeo.json', 'pg28556_mapeo.json', 'pg29870_mapeo.json', 'pg30254_mapeo.json', 'pg3207_mapeo.json', 'pg33283_m